# Starbucks Capstone Challenge

### Introduction

This data set contains simulated data that mimics customer behavior on the Starbucks rewards mobile app. Once every few days, Starbucks sends out an offer to users of the mobile app. An offer can be merely an advertisement for a drink or an actual offer such as a discount or BOGO (buy one get one free). Some users might not receive any offer during certain weeks. 

Not all users receive the same offer, and that is the challenge to solve with this data set.

Your task is to combine transaction, demographic and offer data to determine which demographic groups respond best to which offer type. This data set is a simplified version of the real Starbucks app because the underlying simulator only has one product whereas Starbucks actually sells dozens of products.

Every offer has a validity period before the offer expires. As an example, a BOGO offer might be valid for only 5 days. You'll see in the data set that informational offers have a validity period even though these ads are merely providing information about a product; for example, if an informational offer has 7 days of validity, you can assume the customer is feeling the influence of the offer for 7 days after receiving the advertisement.

You'll be given transactional data showing user purchases made on the app including the timestamp of purchase and the amount of money spent on a purchase. This transactional data also has a record for each offer that a user receives as well as a record for when a user actually views the offer. There are also records for when a user completes an offer. 

Keep in mind as well that someone using the app might make a purchase through the app without having received an offer or seen an offer.

### Example

To give an example, a user could receive a discount offer buy 10 dollars get 2 off on Monday. The offer is valid for 10 days from receipt. If the customer accumulates at least 10 dollars in purchases during the validity period, the customer completes the offer.

However, there are a few things to watch out for in this data set. Customers do not opt into the offers that they receive; in other words, a user can receive an offer, never actually view the offer, and still complete the offer. For example, a user might receive the "buy 10 dollars get 2 dollars off offer", but the user never opens the offer during the 10 day validity period. The customer spends 15 dollars during those ten days. There will be an offer completion record in the data set; however, the customer was not influenced by the offer because the customer never viewed the offer.

### Cleaning

This makes data cleaning especially important and tricky.

You'll also want to take into account that some demographic groups will make purchases even if they don't receive an offer. From a business perspective, if a customer is going to make a 10 dollar purchase without an offer anyway, you wouldn't want to send a buy 10 dollars get 2 dollars off offer. You'll want to try to assess what a certain demographic group will buy when not receiving any offers.

### Final Advice

Because this is a capstone project, you are free to analyze the data any way you see fit. For example, you could build a machine learning model that predicts how much someone will spend based on demographics and offer type. Or you could build a model that predicts whether or not someone will respond to an offer. Or, you don't need to build a machine learning model at all. You could develop a set of heuristics that determine what offer you should send to each customer (i.e., 75 percent of women customers who were 35 years old responded to offer A vs 40 percent from the same demographic to offer B, so send offer A).

# Data Sets

The data is contained in three files:

* portfolio.json - containing offer ids and meta data about each offer (duration, type, etc.)
* profile.json - demographic data for each customer
* transcript.json - records for transactions, offers received, offers viewed, and offers completed

Here is the schema and explanation of each variable in the files:

**portfolio.json**
* id (string) - offer id
* offer_type (string) - type of offer ie BOGO, discount, informational
* difficulty (int) - minimum required spend to complete an offer
* reward (int) - reward given for completing an offer
* duration (int) - time for offer to be open, in days
* channels (list of strings)

**profile.json**
* age (int) - age of the customer 
* became_member_on (int) - date when customer created an app account
* gender (str) - gender of the customer (note some entries contain 'O' for other rather than M or F)
* id (str) - customer id
* income (float) - customer's income

**transcript.json**
* event (str) - record description (ie transaction, offer received, offer viewed, etc.)
* person (str) - customer id
* time (int) - time in hours since start of test. The data begins at time t=0
* value - (dict of strings) - either an offer id or transaction amount depending on the record


## 0. Initiate {zero}

In [98]:
import pandas as pd
import numpy as np
import math
import json
from sklearn.preprocessing import MultiLabelBinarizer 
%matplotlib inline

# read in the json files
portfolio = pd.read_json('data/portfolio.json', orient='records', lines=True)
profile = pd.read_json('data/profile.json', orient='records', lines=True)
transcript = pd.read_json('data/transcript.json', orient='records', lines=True)

## 1. Clean and Organize Data

In [129]:
def clean_and_organize_data(portfolio=portfolio,profile=profile,transcript=transcript):
    ## Portfolio
    portfolio.rename(columns={'id':'offer_id'},inplace=True) # match other table
    # one hot encode channels
    mlb = MultiLabelBinarizer() 
    portfolio = portfolio.join(pd.DataFrame(mlb.fit_transform(portfolio.pop('channels')),
                          columns=mlb.classes_,
                          index=portfolio.index))
    portfolio = portfolio.merge(pd.get_dummies(portfolio['offer_type'],prefix='offer',prefix_sep='_x_'),left_index=True,right_index=True)
    portfolio.drop(columns='offer_type',inplace=True)
    portfolio.set_index('offer_id')
    ## Profile
    profile = profile.dropna(axis = 0)
    profile.loc[:,'became_member_on']=pd.to_datetime(profile['became_member_on'],format='%Y%m%d')
    profile = profile.merge(pd.get_dummies(profile['gender'],prefix='gender',prefix_sep='_x_'),left_index=True,right_index=True)
    profile = profile.drop(columns='gender')
    #profile.set_index('id',inplace=True)
    ## Transcript
    # expand dict in value to its own columns per dict key
    transcript = transcript.merge(transcript.value.apply(pd.Series),left_index=True,right_index=True)
    # merge 
    transcript.loc[transcript['offer_id'].isna(),'offer_id'] = transcript[transcript['offer_id'].isna()]['offer id']
    transcript = transcript.merge(pd.get_dummies(transcript['event'],prefix='event',prefix_sep='_x_'),left_index=True, right_index=True)
    transcript.drop(columns = ['offer id','value'],inplace=True)
    transcript = transcript[transcript['person'].isin(profile.id)].reset_index()
    t_offer = transcript[transcript['event']=='offer received']
    t_viewed = transcript[transcript['event']=='offer viewed']
    t_trans = transcript[transcript['event']=='transaction']
    t_compl = transcript[transcript['event']=='offer completed']
    
    return portfolio,profile,transcript,t_offer,t_viewed,t_trans,t_compl

In [100]:
portfolio_new,profile_new,transcript_new,t_offer,t_viewed,t_trans,t_compl = clean_and_organize_data()

C:\Users\thoma\Anaconda3\lib\site-packages\pandas\core\indexing.py:543: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[item] = s


In [104]:
print(portfolio.shape)
print(profile.shape)
print(transcript.shape)

(10, 5)
(17000, 5)
(306534, 4)


In [105]:
print(portfolio_new.shape)
print(profile_new.shape)
print(transcript_new.shape)

(10, 11)
(14825, 7)
(272762, 13)


In [106]:
print(t_offer.shape)
print(t_viewed.shape)
print(t_compl.shape)
print(t_trans.shape)


(66501, 13)
(49860, 13)
(32444, 13)
(123957, 13)


In [133]:
user_offer = t_offer.pivot_table(values = 'event_x_offer received',columns='offer_id',index = 'person',aggfunc='sum')

In [ ]:
#what percentage of offers viewed

for user in t_offer.person.unique():
    for idx,row in t_offer[t_offer.person==user].iterrows():
        
    

In [141]:
test_user = 'c8a71dc7d32c402e85e2eb3cbe0ad68a'  # has 6 offers

In [154]:
next(t_offer[t_offer['person']==test_user].iterrows())[1]['offer_id']

'0b1e1539f2cc45b7b9fa7c272da2e1d7'

In [177]:
t_viewed[t_viewed.person==test_user].offer_id.isin(['0b1e1539f2cc45b7b9fa7c272da2e1d7'])

80355     False
127354    False
193536    False
232320    False
Name: offer_id, dtype: bool

In [174]:
t_viewed[(t_viewed.person==test_user) & (t_viewed.offer_id=='0b1e1539f2cc45b7b9fa7c272da2e1d7')]['time']

Series([], Name: time, dtype: int64)

## 2. Explore Data

In [ ]:
tot_offers = t_offer.shape[0]
tot_viewed = t_viewed.shape[0]
tot_completed = t_compl.shape[0]
tot_num_purchased = t_trans.shape[0]
tot_sum_purch = t_trans.amount.sum()
tot_sum_reward = t_compl.reward.sum()

print('Percentage viewed : {0:.2f}%'.format(100* tot_viewed/tot_offers))
print('Percentage completed : {0:.2f}%'.format(100*tot_completed/tot_offers))
print('Percentage of purchases that were rewarded  : {0:.2f}%'.format(100*tot_completed/tot_num_purchased))

* we want to separate out transactions that were either not replaced to an offer nor not viewed.
* get transations that were not offers and 

In [ ]:
print(t_offer.shape)
t_offer.head()

In [ ]:
print(t_viewed.shape)
t_viewed.head()

In [ ]:
t_offer_view = t_offer.merge(t_viewed,how='outer',left_on=['person','offer_id'],right_on=['person','offer_id'])

In [ ]:
print(t_offer_view.shape)
t_offer_view.head()

In [ ]:
t_offer_view[t_offer_view['event_y'].isna()]

In [ ]:
offered.values

In [ ]:
offer_viewed = np.intersect1d(offered.values,viewed.values).tolist()

In [ ]:
transcript.loc[(transcript['offer_id'].isin(offer_viewed))&(transcript['person'].isin(test_1_ids)),'purc_viewed']=True

In [ ]:
transcript[transcript['person'].isin(test_1_ids)]

In [ ]:
def user_and_option(arr1, arr2):
    #function per user jterrace via Stack Overflow
    arr1_view = arr1.view([('',arr1.dtype)]*arr1.shape[1])
    arr2_view = arr2.view([('',arr2.dtype)]*arr2.shape[1])
    intersected = numpy.intersect1d(arr1_view, arr2_view)
    return intersected.view(arr1.dtype).reshape(-1, arr1.shape[1])